# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ManipulateDataset').getOrCreate()
spark

22/03/10 12:27:07 WARN Utils: Your hostname, DESKTOP-MR1I23T resolves to a loopback address: 127.0.1.1; using 192.168.15.41 instead (on interface eth0)
22/03/10 12:27:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/10 12:27:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [2]:
tweets = spark.read.csv('./data/Rep_vs_Dem_tweets.csv', header=True, inferSchema=True)

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [3]:
tweets.show(5)

+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         null|                null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+
only showing top 5 rows



**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [4]:
tweets.show(5, False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [5]:
tweets.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [6]:
from pyspark.sql.functions import *

t = tweets.select('Party', 'Handle', 'Tweet', regexp_extract('Tweet', '@LatinoLeader', 0).alias('hasHandler')) \
    .where(col('hasHandler') != '').show(5, False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Party   |Handle       |Tweet                                                                                                                                       |hasHandler   |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Democrat|RepDarrenSoto|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|@LatinoLeader|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+



## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [7]:
t = tweets \
    .withColumn('Party', when((col('Party') != 'Republican') & (col('Party') != 'Democrat'), 'Other').otherwise(col('Party')))

t.limit(10).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Other,None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
5,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...
6,Democrat,RepDarrenSoto,RT @EmgageActionFL: Thank you to all who came ...
7,Democrat,RepDarrenSoto,Hurricane Maria left approx $90 billion in dam...
8,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSo...
9,Democrat,RepDarrenSoto,RT @HispanicCaucus: Trump's anti-immigrant pol...


## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [8]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def findAndDeleteLinksFromTweet(tweet_splited: list):
    recomposed_tweet = ""
    if tweet_splited != None:
        for word in tweet_splited:
            if str(word).startswith('http'):
                tweet_splited.remove(word)
            else:
                recomposed_tweet = recomposed_tweet + word + " "
                continue
        recomposed_tweet = recomposed_tweet[:-1]
    return recomposed_tweet

findAndDeleteLinksFromTweet_udf = udf(lambda x : findAndDeleteLinksFromTweet(x), StringType())

In [9]:
df_no_links = tweets.select('*', split('Tweet', ' ').alias('tweet_splited'))

df_no_links = df_no_links.withColumn('recomposed_tweet', findAndDeleteLinksFromTweet_udf('tweet_splited'))
df_no_links.select('Tweet', 'recomposed_tweet').show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |recomposed_tweet                                                                                                                            |
+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |Today,

## 4. Remove any leading or trailing white space in the tweet column

In [10]:
df_trim = tweets.withColumn('Tweet', trim(col('Tweet')))
df_trim.show(5, False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [11]:
df_party_rename = tweets.withColumnRenamed('Party', 'Dem_Rep')
df_party_rename.limit(5).toPandas()

,Dem_Rep,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...


## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [12]:
df_concat = tweets.withColumn('concat', concat_ws(' ', tweets.Party, tweets.Handle))
df_concat.limit(5).toPandas()

,Party,Handle,Tweet,concat
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",Democrat RepDarrenSoto
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,Democrat RepDarrenSoto
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,Democrat RepDarrenSoto
3,"Congress has allocated about $18…""",None,None,"Congress has allocated about $18…"""
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,Democrat RepDarrenSoto


## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [13]:
def concatenateHashtagsFromTweet(tweet_splited: list):
    recomposed_hashtags = ""
    if tweet_splited != None:
        for word in tweet_splited:
            if str(word).startswith('#'):
                recomposed_hashtags = recomposed_hashtags + word + " "
        recomposed_hashtags = recomposed_hashtags[:-1]
    return recomposed_hashtags

concatenateHashtagsFromTweet_udf = udf(lambda x : concatenateHashtagsFromTweet(x), StringType())

In [14]:
df_hashtags = tweets.withColumn('tweets_splited', split(col('Tweet'), ' '))
df_hashtags = df_hashtags.withColumn('hashtags', concatenateHashtagsFromTweet_udf('tweets_splited'))
df_hashtags.limit(5).toPandas()

,Party,Handle,Tweet,tweets_splited,hashtags
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...","[Today,, Senate, Dems, vote, to, #SaveTheInter...",#SaveTheInternet. #NetNeutrality
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,"[RT, @WinterHavenSun:, Winter, Haven, resident...",
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,"[RT, @NBCLatino:, .@RepDarrenSoto, noted, that...",
3,"Congress has allocated about $18…""",None,None,None,
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,"[RT, @NALCABPolicy:, Meeting, with, @RepDarren...",#NALCABPolicy2018.…


# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [15]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [16]:
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

df = df \
        .withColumn("dob", df['dob'].cast(DateType())) \
        .withColumn("visit1", df['visit1'].cast(DateType())) \
        .withColumn("visit2", df['visit2'].cast(DateType())) \
        .withColumn("visit3", df['visit3'].cast(DateType())) \

print(df.printSchema())

df.limit(5).toPandas()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

None


,first_name,last_name,dob,visit1,visit2,visit3
0,Mohammed,Alfasy,1987-04-08,2016-01-07,2017-02-03,2018-03-02
1,Marcy,Wellmaker,1986-04-08,2015-01-07,2017-01-03,2018-01-02
2,Ginny,Ginger,1986-07-10,2014-08-07,2015-02-03,2016-03-02
3,Vijay,Doberson,1988-05-02,2016-01-07,2018-02-03,2018-03-02
4,Orhan,Gelicek,1987-05-11,2016-05-07,2017-01-03,2018-09-02


## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [17]:
df.select(
    mean(datediff('visit2', 'visit1')).alias('between_1_and_2'), \
    mean(datediff('visit3', 'visit2')).alias('between_2_and_3')) \
.toPandas()

,between_1_and_2,between_2_and_3
0,436.571429,284.714286


## 8. Can you calculate the age of each patient?

In [18]:
from datetime import datetime
import math

def getAge(date):
    birthday = datetime.strptime(str(date), '%Y-%m-%d')
    now = datetime.now()

    return (now - birthday).days / 365.0

getAge_udf = udf(lambda x: getAge(x), DoubleType())

In [19]:
df.select('dob', round(getAge_udf('dob'),1).alias('age')).limit(10).toPandas()

,dob,age
0,1987-04-08,34.9
1,1986-04-08,35.9
2,1986-07-10,35.7
3,1988-05-02,33.9
4,1987-05-11,34.9
5,1956-07-06,65.7
6,2017-10-12,4.4


## 9. Can you extract the month from the first visit column and call it "Month"?

In [20]:
df.select('visit1', month('visit1').alias('Month')).limit(5).toPandas()

,visit1,Month
0,2016-01-07,1
1,2015-01-07,1
2,2014-08-07,8
3,2016-01-07,1
4,2016-05-07,5


## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [24]:
supermarket = spark.read.csv('data/supermarket_sales.csv', inferSchema=True, header=True)

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [25]:
supermarket.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [26]:
supermarket.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [29]:
supermarket_fixed = supermarket \
                        .withColumn('Date', to_date(col('Date'), 'MM/dd/yyyy'))

supermarket_fixed.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3


### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [30]:
supermarket_fixed.select('Date', month('Date')).limit(5).toPandas()

,Date,month(Date)
0,2019-01-05,1
1,2019-03-08,3
2,2019-03-03,3
3,2019-01-27,1
4,2019-02-08,2


## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [31]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [40]:
from pyspark.sql.functions import udf
from string import punctuation

def cleanString(row: str):
    row = row.lower()
    row = row.translate(str.maketrans('', '', punctuation))
    row = row.strip()
    return row

cleanString_udf = udf(lambda x: cleanString(x), StringType())

In [43]:
df_values = reviews.select('review_txt', split(cleanString_udf(col('review_txt')), ' ').alias('cleaned_review'))
df_values.toPandas()

,review_txt,cleaned_review
0,Epic. This is the best movie I have EVER seen,"[epic, this, is, the, best, movie, i, have, ev..."
1,"Pretty good, but I would have liked to seen be...","[pretty, good, but, i, would, have, liked, to,..."
2,So so. Casting could have been improved,"[so, so, casting, could, have, been, improved]"
3,The most EPIC movie of the year! Casting was a...,"[the, most, epic, movie, of, the, year, castin..."
4,Solid but I would have liked to see more of th...,"[solid, but, i, would, have, liked, to, see, m..."
5,THE BOMB!!!!!!!,"[the, bomb]"


## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [45]:
df_values.select('review_txt', 'cleaned_review').where(array_contains(df_values['cleaned_review'], 'epic')) \
        .toPandas()

,review_txt,cleaned_review
0,Epic. This is the best movie I have EVER seen,"[epic, this, is, the, best, movie, i, have, ev..."
1,The most EPIC movie of the year! Casting was a...,"[the, most, epic, movie, of, the, year, castin..."


### That's it! Great Job!